In [ ]:
import pathlib
import random

import sys
sys.path.append('../src')
from mediatools import ffmpeg

In [ ]:
all_vids = list(pathlib.Path('music_vids/').rglob('*.mp4'))
#all_vids = list(pathlib.Path('/mnt/HDDStorage/sys/dwhelper/pros/tia_ling').rglob('*.mp4'))
vids = all_vids
vids = list(random.sample(all_vids, k=2))

print(f'{len(all_vids)=}, {len(vids)=}')
compilation = ffmpeg.create_montage(video_files=vids, output_filename='compilation3.mp4', clip_ratio=30, clip_duration=2.0, verbose=False)